# Loading Dependencies

In [1]:
# Dependencies to Visualize the model
import tensorflow as tf
%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
import numpy as np
# Setting seed for reproducibility
np.random.seed(0)

# Filepaths, pandas, numpy, Tensorflow, and scikit-image
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import skimage as sk

2024-04-18 11:34:38.569061: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Stratify Images in Main Image Repository

In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the CSV file
csv_file = "Resources/HAM10000_metadata.csv"
metadata = pd.read_csv(csv_file)

# Define the source directory where all the images are located
source_dir = "Resources/Skin Cancer"

# Define the target directories for train and val splits
train_dir = "Resources/Skin Cancer/train"
val_dir = "Resources/Skin Cancer/val"

# Define the split ratio (e.g., 0.8 for 80% train, 0.2 for 20% val)
split_ratio = 0.8

# Create the target directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Get the unique class labels
class_labels = metadata["dx"].unique()

# Create subdirectories for each class in train and val directories
for label in class_labels:
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(val_dir, label), exist_ok=True)

# Split the metadata into train and validation sets using stratified splitting
# Stratify was required here to ensure that an 80-20 split occured for all classes in the dataset, not just an 80-20 split of the entire dataset.
train_metadata, val_metadata = train_test_split(
    metadata, stratify=metadata["dx"], test_size=1 - split_ratio, random_state=42
)

# Move or copy the images to the respective train and validation directories
for _, row in train_metadata.iterrows():
    image_id = row["image_id"]
    image_path = os.path.join(source_dir, f"{image_id}.jpg")
    class_label = row["dx"]
    target_dir = os.path.join(train_dir, class_label)
    shutil.copy(image_path, target_dir)

for _, row in val_metadata.iterrows():
    image_id = row["image_id"]
    image_path = os.path.join(source_dir, f"{image_id}.jpg")
    class_label = row["dx"]
    target_dir = os.path.join(val_dir, class_label)
    shutil.copy(image_path, target_dir)

print("Stratified splitting and image organization completed.")

# Preprocessing of Images

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to your image directory
image_directory = "../Resources/Skin Cancer"

# Defining original image size
image_size = (600, 450)

# Define the batch size
batch_size = 32
 
# Create an ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
)

# Load and preprocess the train dataset with data augmentation
train_dataset = train_datagen.flow_from_directory(
    directory=os.path.join(image_directory, "train"),  # Use the 'train' directory
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42,
)

# Create an ImageDataGenerator for validation data (no augmentation)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values

# Load and preprocess the validation dataset without data augmentation
val_dataset = val_datagen.flow_from_directory(
    directory=os.path.join(image_directory, "val"),  # Use the 'val' directory
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42,
)

# Print the class names
class_names = list(train_dataset.class_indices.keys())
print("Class Names:", class_names)

# Print the shape of the datasets
print("Train Dataset Shape:", train_dataset.image_shape)
print("Validation Dataset Shape:", val_dataset.image_shape)

Found 8012 images belonging to 7 classes.
Found 2003 images belonging to 7 classes.
Class Names: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
Train Dataset Shape: (600, 450, 3)
Validation Dataset Shape: (600, 450, 3)


# Class Weight Adjustment
This section is to adjust the loss function to more heavily weight our cancerous classes - 'mel', 'akeic', and 'bbc'. This will account for the class imbalance in our original data sets to give us more accurate predictions on the cancerous classes. 

In [10]:
# Importing Dependencies
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger

import csv
import os

from sklearn.utils import class_weight

In [11]:
# Creating a CSV file to store the model results
csv_file = "model_results.csv"
fieldnames = [
    "architecture",
    "optimizer",
    "loss",
    "accuracy",
    "precision",
    "recall",
    "auc",
]

# Write the header to the CSV file. Data is written to the file after model_result
with open(csv_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

# Define batch size, architecture, and optimizer
batch_size = 32
architectures = ["InceptionV3"]
optimizers = ["Adam"]

# Create directory to store models
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

# Initialize model_results to store the evaluation results
model_results = []

In [21]:
from sklearn.utils import class_weight

# Initialize an empty dictionary to store the adjusted class weights
adjusted_class_weights = {}

# Loop through architectures and optimizers
for architecture in architectures:
    for optimizer_name in optimizers:
        # Define the optimizer
        if optimizer_name == "Adam":
            optimizer = Adam(learning_rate=0.001)
        elif optimizer_name == "RMSprop":
            optimizer = RMSprop(learning_rate=0.001)
        elif optimizer_name == "SGD":
            optimizer = SGD(learning_rate=0.001, momentum=0.9)

        # Load and preprocess data using tf.keras.preprocessing
        train_datagen = ImageDataGenerator(
            preprocessing_function=preprocessing_function,
        )

        train_dataset = train_datagen.flow_from_directory(
            "../Resources/Skin Cancer/train",
            target_size=input_shape[:2],
            batch_size=batch_size,
            class_mode="categorical",
        )

        # Calculate class weights
        class_weights = class_weight.compute_class_weight(
            class_weight='balanced',
            classes=np.unique(train_dataset.classes),
            y=train_dataset.classes
        )
        class_weight_dict = dict(enumerate(class_weights))

        print("Original Class Weights:", class_weights)
        print("Original Class Weight Dictionary:", class_weight_dict)

        # Define the indices of the classes you want to prioritize
        priority_classes = [0,1,4] # Class numbers for 'akeic', 'bcc', and 'mel' respectively

        # Define the weight multiplier for the priority classes
        priority_weight_multiplier = 5  # Adjust this value as needed

        # Assign higher weights to the priority classes and keep the weights for other classes unchanged
        for class_num, class_weight in enumerate(class_weights):
            if class_num in priority_classes:
                if class_num == 4:  # 'mel' is class 4
                    class_weight_dict[class_num] = class_weight * priority_weight_multiplier * 4  # Quadrupling the multiplier for 'mel'
                else:
                    class_weight_dict[class_num] = class_weight * priority_weight_multiplier
            else:
                class_weight_dict[class_num] = class_weight
        
        # Add adjusted class weights to the dictionary
        adjusted_class_weights.update(class_weight_dict)

        # Define the loss function with the manually adjusted class weights
        def weighted_categorical_crossentropy(y_true, y_pred):
            """
            Custom loss function for weighted categorical crossentropy.

            Args:
                y_true (tensor): True labels.
                y_pred (tensor): Predicted probabilities.

            Returns:
                tensor: Weighted categorical crossentropy loss.
            """

            # Define a tensor containing the class weights
            weights = tf.constant([adjusted_class_weights[class_num] for class_num in range(num_classes)], dtype=tf.float32)

            # Clip predicted probabilities to avoid numerical instability
            y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())

            # Compute the categorical cross-entropy loss with weighted class probabilities
            loss = tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred) * weights, axis=-1))

            return loss

        # Define the pre-trained model architecture
        if architecture == "VGG16":
            base_model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
        elif architecture == "ResNet50":
            base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
        elif architecture == "InceptionV3":
            base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=input_shape)

        # Freeze the layers of the pre-trained model
        for layer in base_model.layers:
            layer.trainable = False

        # Add custom layers on top of the pre-trained model
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, activation="relu")(x)
        x = Dropout(0.5)(x)
        predictions = Dense(num_classes, activation="softmax")(x)

        # Create the final model
        model = Model(inputs=base_model.input, outputs=predictions)

        # Compile the model with the weighted loss function
        model.compile(
            optimizer=optimizer,
            loss=weighted_categorical_crossentropy,
            metrics=["accuracy", Precision(), Recall(), AUC()],
        )

Found 8012 images belonging to 7 classes.
Original Class Weights: [ 4.36859324  2.78484532  1.30212904 12.44099379  1.28603531  0.21338021
 10.04010025]
Original Class Weight Dictionary: {0: 4.368593238822246, 1: 2.7848453249913105, 2: 1.3021290427433772, 3: 12.440993788819876, 4: 1.2860353130016051, 5: 0.21338020666879728, 6: 10.040100250626567}


In [23]:
# Print adjusted class weights
print("Adjusted Class Weights:")
for class_num, class_weight in adjusted_class_weights.items():
    print(f"{class_num}: {class_weight}")

#Print adjusted class weights dictionary
print(adjusted_class_weights)

Adjusted Class Weights:
0: 21.84296619411123
1: 13.924226624956553
2: 1.3021290427433772
3: 12.440993788819876
4: 25.720706260032102
5: 0.21338020666879728
6: 10.040100250626567
{0: 21.84296619411123, 1: 13.924226624956553, 2: 1.3021290427433772, 3: 12.440993788819876, 4: 25.720706260032102, 5: 0.21338020666879728, 6: 10.040100250626567}


# Step-by-step Model Building
This section contains a cell-by-cell break down of the model creation, and allows for the testing of singular models

In [ ]:
# Importing Dependencies
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau
import os
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger
import csv

### Choose an Architecture

In [ ]:
# Choose an Architecture
architecture = "InceptionV3"  # Options: "VGG16", "ResNet50", "InceptionV3"

# Set the input shape and preprocessing function based on the selected architecture
if architecture == "VGG16":
    input_shape = (224, 224, 3)
    preprocessing_function = tf.keras.applications.vgg16.preprocess_input
if architecture == "resnet50":
    input_shape = (224, 224, 3)
    preprocessing_function = tf.keras.applications.resnet.preprocess_input
elif architecture == "InceptionV3":
    input_shape = (299, 299, 3)
    preprocessing_function = tf.keras.applications.inception_v3.preprocess_input

In [ ]:
# Load and preprocess data using tf.keras.preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocessing_function,
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # horizontal_flip=True,
)

train_dataset = train_datagen.flow_from_directory(
    "Resources/Skin Cancer/train",
    target_size=input_shape[:2],
    batch_size=32,
    class_mode="categorical",
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

val_dataset = val_datagen.flow_from_directory(
    "Resources/Skin Cancer/val",
    target_size=input_shape[:2],
    batch_size=32,
    class_mode="categorical",
)

### Set base_model

In [ ]:
# Define the pre-trained model architecture
if architecture == "VGG16":
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
elif architecture == "ResNet50":
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
elif architecture == "InceptionV3":
    base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=input_shape)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

### Custom Layering 

In [ ]:
# Get the number of unique classes from the train_dataset
num_classes = len(train_dataset.class_indices)

# Add custom layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
num_classes = len(train_dataset.class_indices)
predictions = Dense(num_classes, activation="softmax")(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

### Choose an Optimizer

In [ ]:
# Choose an Optimizer
optimizer_name = "Adam"  # Options: "Adam", "RMSprop", "SGD"

if optimizer_name == "Adam":
    optimizer = Adam(learning_rate=0.001)
elif optimizer_name == "RMSprop":
    optimizer = RMSprop(learning_rate=0.001)
elif optimizer_name == "SGD":
    optimizer = SGD(learning_rate=0.001, momentum=0.9)

lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, verbose=1)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy", Precision(), Recall(), AUC()],
)

### Create Callbacks for TensorBoard and CSV tracking

In [ ]:
# Create callbacks
tensorboard_callback = TensorBoard(log_dir=f"./logs/{architecture}_{optimizer_name}", histogram_freq=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=f"models/model_{architecture}_{optimizer_name}.weights.h5",
    save_weights_only=True,
    save_best_only=True,
    monitor="val_loss",
    verbose=1,
)
csv_logger = CSVLogger(f"models/model_{architecture}_{optimizer_name}_training.csv")

### Run Model

In [ ]:
# Train the model with callbacks
epochs = 10
history = model.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // 32,
    validation_data=val_dataset,
    validation_steps=val_dataset.samples // 32,
    epochs=epochs,
    callbacks=[tensorboard_callback, lr_scheduler, checkpoint_callback, csv_logger],
)

### Evaluate Model

In [ ]:
# Evaluate the model on the validation set and print the results
loss, accuracy, precision, recall, auc = model.evaluate(val_dataset)
print(f"Model: {architecture}, Optimizer: {optimizer_name}")
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation AUC-ROC: {auc:.4f}")

# Save the model
model.save(f"models/model_{architecture}_{optimizer_name}.h5")

# Continuous Testing 
This section contains a conglomerated model run. It was created so that the models could be tested overnight. I'm going to bed now.

In [24]:
# Importing Dependencies
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger

import csv
import os

from sklearn.utils import class_weight

# Run Number - use to create new directory or add to existing directory
run_number = 7
run_dir = f"run{run_number}"
os.makedirs(run_dir, exist_ok=True)

# Creating a CSV file to store the model results
csv_file = "model_results.csv"
fieldnames = [
    "architecture",
    "optimizer",
    "loss",
    "accuracy",
    "precision",
    "recall",
    "auc",
]

# Write the header to the CSV file. Data is written to the file after model_result
with open(csv_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

batch_size = 32
architectures = ["InceptionV3"] # Options: "VGG16", "ResNet50", "InceptionV3"
optimizers = ["Adam"] # Options: "Adam", "RMSprop", "SGD"

# Create directory to store models
models_dir = f"{run_dir}/models"
os.makedirs(models_dir, exist_ok=True)

# Initialize model_results to store the evaluation results
model_results = []

for architecture in architectures:
    # Set the input shape and preprocessing function based on the selected architecture
    if architecture == "VGG16" or architecture == "ResNet50":
        input_shape = (224, 224, 3)
        preprocessing_function = tf.keras.applications.vgg16.preprocess_input
    elif architecture == "ResNet50":
        input_shape = (224, 224, 3)
        preprocessing_function = tf.keras.applications.resnet50.preprocess_input
    elif architecture == "InceptionV3":
        input_shape = (299, 299, 3)
        preprocessing_function = tf.keras.applications.inception_v3.preprocess_input

    # Load and preprocess data using tf.keras.preprocessing
    # This adds data augmentation to the training dataset
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocessing_function,
    )

    train_dataset = train_datagen.flow_from_directory(
        "../Resources/Skin Cancer/train",
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode="categorical",
    )

    
    val_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

    # Define the validation dataset without data augmentation
    val_dataset = val_datagen.flow_from_directory(
        "../Resources/Skin Cancer/val",
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode="categorical",
    )

    # Define the pre-trained model architecture. Will select proper model based on current 'architecture' in for loop
    if architecture == "VGG16":
        base_model = VGG16(
            weights="imagenet", include_top=False, input_shape=input_shape
        )
    elif architecture == "ResNet50":
        base_model = ResNet50(
            weights="imagenet", include_top=False, input_shape=input_shape
        )
    elif architecture == "InceptionV3":
        base_model = InceptionV3(
            weights="imagenet", include_top=False, input_shape=input_shape
        )

    # Freeze the layers of the pre-trained model
    for layer in base_model.layers:
        layer.trainable = False

    # Get the number of unique classes from the train_dataset
    num_classes = len(train_dataset.class_indices)

    # Add custom layers on top of the pre-trained model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation="softmax")(x)

    # Create the final model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Step through and adjust for each optimizer
    for optimizer_name in optimizers:
        # Define the optimizer
        if optimizer_name == "Adam":
            optimizer = Adam(learning_rate=0.001)
        elif optimizer_name == "RMSprop":
            optimizer = RMSprop(learning_rate=0.001)
        elif optimizer_name == "SGD":
            optimizer = SGD(learning_rate=0.001, momentum=0.9)

        # Create learning rate scheduler that monitors validation loss
        lr_scheduler = ReduceLROnPlateau(
            monitor="val_loss", factor=0.1, patience=5, verbose=1
        )

        # Compile the model with the weighted loss function
        model.compile(
            optimizer=optimizer,
            loss=weighted_categorical_crossentropy,  # Use the custom loss function
            metrics=["accuracy", Precision(), Recall(), AUC()],
        )

        # Create a TensorBoard callback with a separate log directory for each model and optimizer
        tensorboard_callback = TensorBoard(
            log_dir=f"./{run_dir}/logs/{architecture}_{optimizer_name}", histogram_freq=1
        )

        # Create an EarlyStopping callback to prevent overfitting
        early_stopping = EarlyStopping(
            monitor="val_loss", patience=5, restore_best_weights=True
        )

        # Create a ModelCheckpoint callback to save the best model weights
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(
                models_dir, f"model_{architecture}_{optimizer_name}.weights.h5"
            ),
            save_weights_only=True,
            save_best_only=True,
            monitor="val_loss",
            verbose=1,
        )

        # Create a CSVLogger callback with keras
        csv_logger = CSVLogger(
            os.path.join(
                models_dir, f"model_{architecture}_{optimizer_name}_training.csv"
            )
        )

        # IMPORTANT: RESEARACH CLASS WEIGHTING
        # Calculate class weights
        # This step is required, as some of the classes are imbalanced, in particular, the val classes
        # class_weights = class_weight.compute_class_weight(
        #     'balanced',
        #     classes=np.unique(train_dataset.classes),
        #     y=train_dataset.classes
        # )
        # class_weight_dict = dict(enumerate(class_weights))

        # Train the model with the checkpoint callback and weights
        epochs = 10
        history = model.fit(
            train_dataset,
            steps_per_epoch=train_dataset.samples // batch_size,
            validation_data=val_dataset,
            validation_steps=val_dataset.samples // batch_size,
            epochs=epochs,
            class_weight=adjusted_class_weights,
            callbacks=[tensorboard_callback, lr_scheduler, checkpoint_callback, csv_logger],
        )

        # Save the optimizer state to avoid retraining the model
        model.save(
            os.path.join(models_dir, f"model_{architecture}_{optimizer_name}.h5")
        )

        # Evaluate the model on the validation set and print the results
        loss, accuracy, precision, recall, auc = model.evaluate(val_dataset)
        print(f"Model: {architecture}, Optimizer: {optimizer_name}")
        print(f"Validation Loss: {loss:.4f}")
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(f"Validation Precision: {precision:.4f}")
        print(f"Validation Recall: {recall:.4f}")
        print(f"Validation AUC-ROC: {auc:.4f}")

        # Append the model results to the list
        model_result = {
            "architecture": architecture,
            "optimizer": optimizer_name,
            "loss": loss,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "auc": auc,
        }
        model_results.append(model_result)

        # Write the current model result to the CSV file
        with open(csv_file, mode="a", newline="") as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writerow(model_result)

        # Save the model
        model.save(
            os.path.join(models_dir, f"model_{architecture}_{optimizer_name}.h5")
        )

Found 8012 images belonging to 7 classes.
Found 2003 images belonging to 7 classes.
Epoch 1/10


2024-04-18 13:24:40.677304: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


250/250 [==============================] - ETA: 0s - loss: 35.1192 - accuracy: 0.1189 - precision_8: 0.1307 - recall_8: 0.0940 - auc_8: 0.4999

2024-04-18 13:33:40.930077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_loss improved from inf to 4.84906, saving model to run7/models/model_InceptionV3_Adam.weights.h5
250/250 [==============================] - 679s 3s/step - loss: 35.1192 - accuracy: 0.1189 - precision_8: 0.1307 - recall_8: 0.0940 - auc_8: 0.4999 - val_loss: 4.8491 - val_accuracy: 0.1265 - val_precision_8: 0.1371 - val_recall_8: 0.0948 - val_auc_8: 0.6237 - lr: 0.0010
Epoch 2/10
250/250 [==============================] - ETA: 0s - loss: 25.0361 - accuracy: 0.1511 - precision_8: 0.1568 - recall_8: 0.1059 - auc_8: 0.6103
Epoch 2: val_loss improved from 4.84906 to 4.61738, saving model to run7/models/model_InceptionV3_Adam.weights.h5
250/250 [==============================] - 688s 3s/step - loss: 25.0361 - accuracy: 0.1511 - precision_8: 0.1568 - recall_8: 0.1059 - auc_8: 0.6103 - val_loss: 4.6174 - val_accuracy: 0.1386 - val_precision_8: 0.1451 - val_recall_8: 0.1200 - val_auc_8: 0.5983 - lr: 0.0010
Epoch 3/10
250/250 [==============================] - ETA: 0s - loss: 23.8224

2024-04-18 15:12:35.685121: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


63/63 [==============================] - 121s 2s/step - loss: 4.0313 - accuracy: 0.2771 - precision_8: 0.2657 - recall_8: 0.2022 - auc_8: 0.7654
Model: InceptionV3, Optimizer: Adam
Validation Loss: 4.0313
Validation Accuracy: 0.2771
Validation Precision: 0.2657
Validation Recall: 0.2022
Validation AUC-ROC: 0.7654


In [25]:
# Print adjusted class weights
print("Adjusted Class Weights:")
for class_num, class_weight in adjusted_class_weights.items():
    print(f"{class_num}: {class_weight}")

#Print adjusted class weights dictionary
print(adjusted_class_weights)

Adjusted Class Weights:
0: 21.84296619411123
1: 13.924226624956553
2: 1.3021290427433772
3: 12.440993788819876
4: 25.720706260032102
5: 0.21338020666879728
6: 10.040100250626567
{0: 21.84296619411123, 1: 13.924226624956553, 2: 1.3021290427433772, 3: 12.440993788819876, 4: 25.720706260032102, 5: 0.21338020666879728, 6: 10.040100250626567}


# Class Weight Testing

In [4]:
# Importing Dependencies
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger

import csv
import os

from sklearn.utils import class_weight

# Creating a CSV file to store the model results
csv_file = "model_results.csv"
fieldnames = [
    "architecture",
    "optimizer",
    "loss",
    "accuracy",
    "precision",
    "recall",
    "auc",
]

# Write the header to the CSV file. Data is written to the file after model_result
with open(csv_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

batch_size = 32
architectures = ["InceptionV3"]
optimizers = ["Adam"]

# Create directory to store models
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

# Initialize model_results to store the evaluation results
model_results = []

for architecture in architectures:
    # Set the input shape and preprocessing function based on the selected architecture
    if architecture == "VGG16" or architecture == "ResNet50":
        input_shape = (224, 224, 3)
        preprocessing_function = tf.keras.applications.vgg16.preprocess_input
    elif architecture == "InceptionV3":
        input_shape = (299, 299, 3)
        preprocessing_function = tf.keras.applications.inception_v3.preprocess_input

    # Load and preprocess data using tf.keras.preprocessing
    # This is add data augmentation to the training dataset
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocessing_function,
        # rotation_range=20,
        # width_shift_range=0.2,
        # height_shift_range=0.2,
        # horizontal_flip=True,
    )

    train_dataset = train_datagen.flow_from_directory(
        "Resources/Skin Cancer/train",
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode="categorical",
    )

    
    val_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

    # Define the validation dataset without data augmentation
    val_dataset = val_datagen.flow_from_directory(
        "Resources/Skin Cancer/val",
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode="categorical",
    )

    # Define the pre-trained model architecture. Will select proper model based on current 'architecture' in for loop
    if architecture == "VGG16":
        base_model = VGG16(
            weights="imagenet", include_top=False, input_shape=input_shape
        )
    elif architecture == "ResNet50":
        base_model = ResNet50(
            weights="imagenet", include_top=False, input_shape=input_shape
        )
    elif architecture == "InceptionV3":
        base_model = InceptionV3(
            weights="imagenet", include_top=False, input_shape=input_shape
        )

    # Freeze the layers of the pre-trained model
    for layer in base_model.layers:
        layer.trainable = False

    # Get the number of unique classes from the train_dataset
    num_classes = len(train_dataset.class_indices)

    # Add custom layers on top of the pre-trained model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation="softmax")(x)

    # Create the final model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Step through and adjust for each optimizer
    for optimizer_name in optimizers:
        # Define the optimizer
        # Learning rate is the size of the step the optimizer will take to minimize the loss function
        if optimizer_name == "Adam":
            optimizer = Adam(learning_rate=0.001)
        elif optimizer_name == "RMSprop":
            optimizer = RMSprop(learning_rate=0.001)
        elif optimizer_name == "SGD":
            optimizer = SGD(learning_rate=0.001, momentum=0.9)

        # Create learning rate scheduler that monitors validation loss
        lr_scheduler = ReduceLROnPlateau(
            monitor="val_loss", factor=0.1, patience=5, verbose=1
        )

        # Compile the model
        model.compile(
            optimizer=optimizer,
            loss="categorical_crossentropy",
            metrics=["accuracy", Precision(), Recall(), AUC()],
        )

        # Create a TensorBoard callback with a separate log directory for each model and optimizer
        tensorboard_callback = TensorBoard(
            log_dir=f"./logs/{architecture}_{optimizer_name}", histogram_freq=1
        )

        # Create an EarlyStopping callback to prevent overfitting
        early_stopping = EarlyStopping(
            monitor="val_loss", patience=5, restore_best_weights=True
        )

        # Create a ModelCheckpoint callback to save the best model weights
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(
                models_dir, f"model_{architecture}_{optimizer_name}.weights.h5"
            ),
            save_weights_only=True,
            save_best_only=True,
            monitor="val_loss",
            verbose=1,
        )

        # Create a CSVLogger callback with keras
        csv_logger = CSVLogger(
            os.path.join(
                models_dir, f"model_{architecture}_{optimizer_name}_training.csv"
            )
        )

        # Calculate class weights
        # This step is required, as some of the classes are imbalanced, in particular, the val classes
        class_weights = class_weight.compute_class_weight(
            'balanced',
            classes=np.unique(train_dataset.classes),
            y=train_dataset.classes
        )
        class_weight_dict = dict(enumerate(class_weights))

        

Found 8012 images belonging to 7 classes.
Found 2003 images belonging to 7 classes.
87910968/87910968 [==============================] - 26s 0us/step


In [6]:
print(class_weights)
print(class_weight_dict)

for class_name, class_weight in zip(class_names, class_weights):
    print(f"{class_name}: {class_weight}")

[ 4.36859324  2.78484532  1.30212904 12.44099379  1.28603531  0.21338021
 10.04010025]
{0: 4.368593238822246, 1: 2.7848453249913105, 2: 1.3021290427433772, 3: 12.440993788819876, 4: 1.2860353130016051, 5: 0.21338020666879728, 6: 10.040100250626567}
akiec: 4.368593238822246
bcc: 2.7848453249913105
bkl: 1.3021290427433772
df: 12.440993788819876
mel: 1.2860353130016051
nv: 0.21338020666879728
vasc: 10.040100250626567


### Play with adjusting the loss function for 'akiec', 'bcc', and 'mel' to address class imbalance

In [17]:
# Importing Dependencies
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger

import csv
import os

from sklearn.utils import class_weight

In [18]:
# Creating a CSV file to store the model results
csv_file = "model_results.csv"
fieldnames = [
    "architecture",
    "optimizer",
    "loss",
    "accuracy",
    "precision",
    "recall",
    "auc",
]

# Write the header to the CSV file. Data is written to the file after model_result
with open(csv_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

# Define batch size, architecture, and optimizer
batch_size = 32
architectures = ["InceptionV3"]
optimizers = ["Adam"]

# Create directory to store models
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

# Initialize model_results to store the evaluation results
model_results = []

In [29]:
from sklearn.utils import class_weight

# Loop through architectures and optimizers
for architecture in architectures:
    for optimizer_name in optimizers:
        # Define the optimizer
        if optimizer_name == "Adam":
            optimizer = Adam(learning_rate=0.001)
        elif optimizer_name == "RMSprop":
            optimizer = RMSprop(learning_rate=0.001)
        elif optimizer_name == "SGD":
            optimizer = SGD(learning_rate=0.001, momentum=0.9)

        # Load and preprocess data using tf.keras.preprocessing
        train_datagen = ImageDataGenerator(
            preprocessing_function=preprocessing_function,
        )

        train_dataset = train_datagen.flow_from_directory(
            "Resources/Skin Cancer/train",
            target_size=input_shape[:2],
            batch_size=batch_size,
            class_mode="categorical",
        )

        # Calculate class weights
        class_weights = class_weight.compute_class_weight(
            class_weight='balanced',
            classes=np.unique(train_dataset.classes),
            y=train_dataset.classes
        )
        class_weight_dict = dict(enumerate(class_weights))

        print("Class Weights:", class_weights)
        print("Class Weight Dictionary:", class_weight_dict)

        # Define the indices of the classes you want to prioritize
        priority_classes = ['mel', 'bcc', 'akiec']

        # Define the weight multiplier for the priority classes
        priority_weight_multiplier = 5  # Adjust this value as needed

        # Assign higher weights to the priority classes and keep the weights for other classes unchanged
        for class_name, class_weight in zip(class_names, class_weights):
            if class_name in priority_classes:
                if class_name == 'mel':
                    class_weight_dict[class_name] = class_weight * priority_weight_multiplier * 4  # Quadrulpling the multiplier for 'mel'
                else:
                    class_weight_dict[class_name] = class_weight * priority_weight_multiplier
            else:
                class_weight_dict[class_name] = class_weight

        # Define the loss function with the manually adjusted class weights
        def weighted_categorical_crossentropy(y_true, y_pred):
            """
            Custom loss function for weighted categorical crossentropy.

            Args:
                y_true (tensor): True labels.
                y_pred (tensor): Predicted probabilities.

            Returns:
                tensor: Weighted categorical crossentropy loss.
            """

            # Define a tensor containing the class weights
            weights = tf.constant([class_weight_dict[class_name] for class_name in class_names], dtype=tf.float32)

            # Clip predicted probabilities to avoid numerical instability
            y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())

            # Compute the categorical cross-entropy loss with weighted class probabilities
            loss = tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred) * weights, axis=-1))

            return loss

        # Define the pre-trained model architecture
        if architecture == "VGG16":
            base_model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
        elif architecture == "ResNet50":
            base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
        elif architecture == "InceptionV3":
            base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=input_shape)

        # Freeze the layers of the pre-trained model
        for layer in base_model.layers:
            layer.trainable = False

        # Add custom layers on top of the pre-trained model
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, activation="relu")(x)
        x = Dropout(0.5)(x)
        predictions = Dense(num_classes, activation="softmax")(x)

        # Create the final model
        model = Model(inputs=base_model.input, outputs=predictions)

        # Compile the model with the weighted loss function
        model.compile(
            optimizer=optimizer,
            loss=weighted_categorical_crossentropy,
            metrics=["accuracy", Precision(), Recall(), AUC()],
        )

Found 8012 images belonging to 7 classes.
Class Weights: [ 4.36859324  2.78484532  1.30212904 12.44099379  1.28603531  0.21338021
 10.04010025]
Class Weight Dictionary: {0: 4.368593238822246, 1: 2.7848453249913105, 2: 1.3021290427433772, 3: 12.440993788819876, 4: 1.2860353130016051, 5: 0.21338020666879728, 6: 10.040100250626567}


In [30]:
# Print original class weights
for class_name, class_weight in zip(class_names, class_weights):
    print(f"{class_name}: {class_weight}")

akiec: 4.368593238822246
bcc: 2.7848453249913105
bkl: 1.3021290427433772
df: 12.440993788819876
mel: 1.2860353130016051
nv: 0.21338020666879728
vasc: 10.040100250626567


In [31]:
# Print adjusted class weights
for class_name, class_weight in class_weight_dict.items():
    print(f"{class_name}: {class_weight}")

0: 4.368593238822246
1: 2.7848453249913105
2: 1.3021290427433772
3: 12.440993788819876
4: 1.2860353130016051
5: 0.21338020666879728
6: 10.040100250626567
akiec: 21.84296619411123
bcc: 13.924226624956553
bkl: 1.3021290427433772
df: 12.440993788819876
mel: 25.720706260032102
nv: 0.21338020666879728
vasc: 10.040100250626567


# Resume from Interrupt 
Should the overnight run need to be interupted, it is possible to pick up where the interruption left off, and continue training the model

In [ ]:
# Load the model architecture and optimizer
architecture = "VGG16"  # Specify the architecture you want to resume training for
optimizer_name = "Adam"  # Specify the optimizer you want to resume training with

# Load the model 
model = load_model(os.path.join(models_dir, f"model_{architecture}_{optimizer_name}.h5"))

# Load the optimizer state
with open(os.path.join(models_dir, f"model_{architecture}_{optimizer_name}_optimizer_state.pkl"), 'rb') as f:
    optimizer_weights = model.load(f)
    model.optimizer.set_weights(optimizer_weights)

# Resume training from a specific epoch
resume_epoch = 5  # Specify the epoch number to resume from
for epoch in range(resume_epoch, epochs):
    history = model.fit(
        train_dataset,
        steps_per_epoch=train_dataset.samples // 32,
        validation_data=val_dataset,
        validation_steps=val_dataset.samples // 32,
        epochs=1,
        callbacks=[tensorboard_callback, lr_scheduler, checkpoint_callback],
        initial_epoch=epoch
    )

    # Save the optimizer state
    with open(os.path.join(models_dir, f"model_{architecture}_{optimizer_name}_optimizer_state.pkl"), 'wb') as f:
        model.dump(model.optimizer.get_weights(), f)

     # Save the optimizer state
    with open(
                os.path.join(
                    models_dir,
                    f"model_{architecture}_{optimizer_name}_optimizer_state.pkl",
                ),
                "wb",
            ) as f:
                model.dump(model.optimizer.get_weights(), f)

        # Evaluate the model on the validation set
        loss, accuracy, precision, recall, auc = model.evaluate(val_dataset)
        print(f"Model: {architecture}, Optimizer: {optimizer_name}")
        print(f"Validation Loss: {loss:.4f}")
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(f"Validation Precision: {precision:.4f}")
        print(f"Validation Recall: {recall:.4f}")
        print(f"Validation AUC-ROC: {auc:.4f}")

        # Append the model results to the list
        model_results.append(
            {
                "architecture": architecture,
                "optimizer": optimizer_name,
                "loss": loss,
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "auc": auc,
            }
        )
        # Save the model
        model.save(
            os.path.join(models_dir, f"model_{architecture}_{optimizer_name}.h5")
        )


# Evaluating and Visualizing the Data

In [32]:
# Importing Dependencies
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import csv
import os

from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import os

model_results = "model_results.csv"

# Run Number - use to create new directory or add to existing directory
run_number = 7
run_dir = f"run{run_number}"
os.makedirs(run_dir, exist_ok=True)


architectures = ["InceptionV3"] 
optimizers = ["Adam"]

batch_size = 32

# Define the preprocessing function based on the architectures you want to visualize
architectures_to_visualize = ["InceptionV3"]  # Specify the architectures you want to visualize
preprocessing_functions = {
    "VGG16": tf.keras.applications.vgg16.preprocess_input,
    "ResNet50": tf.keras.applications.resnet50.preprocess_input,
    "InceptionV3": tf.keras.applications.inception_v3.preprocess_input
}


# Create a directory to store the visualization results
visualizations_dir = f"{run_dir}/visualizations"
os.makedirs(visualizations_dir, exist_ok=True)

#

# Iterate over each model architecture and optimizer
for architecture in architectures:
    # Set the preprocessing function based on the architecture
    preprocessing_function = preprocessing_functions[architecture]

    # Set the input shape and preprocessing function based on the selected architecture
    if architecture == "VGG16":
        input_shape = (224, 224, 3)
        preprocessing_function = tf.keras.applications.vgg16.preprocess_input
    if architecture == "ResNet50":
        input_shape = (224, 224, 3)
        preprocessing_function = tf.keras.applications.resnet50.preprocess_input
    elif architecture == "InceptionV3":
        input_shape = (299, 299, 3)
        preprocessing_function = tf.keras.applications.inception_v3.preprocess_input

    # Redefine the validation dataset with the corresponding preprocessing function
    val_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)
    val_dataset = val_datagen.flow_from_directory(
        "../Resources/Skin Cancer/val",
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode="categorical",
    )

    # Get the class names from the redefined val_dataset
    class_names = list(val_dataset.class_indices.keys())


    for optimizer_name in optimizers:
        # Load the trained model
        model = load_model(os.path.join(f"models/model_{architecture}_{optimizer_name}.h5"))

        # Make predictions on the validation dataset using the redefined val_dataset
        y_pred = model.predict(val_dataset)
        y_pred_classes = np.argmax(y_pred, axis=1)

        # Get the true labels of the validation dataset
        y_true = val_dataset.classes

        # Compute the confusion matrix
        cm = confusion_matrix(y_true, y_pred_classes)
        print(f"Confusion Matrix for {architecture}_{optimizer_name}:")
        print(cm)

        # Save the confusion matrix as a CSV file
        cm_filename = f"confusion_matrix_{architecture}_{optimizer_name}.csv"
        np.savetxt(os.path.join(visualizations_dir, cm_filename), cm, delimiter=",")

        # Compute the classification report
        cr = classification_report(y_true, y_pred_classes, target_names=class_names)
        print(f"Classification Report for {architecture}_{optimizer_name}:")
        print(cr)

        # Save the classification report as a text file
        cr_filename = f"classification_report_{architecture}_{optimizer_name}.txt"
        with open(os.path.join(visualizations_dir, cr_filename), "w") as file:
            file.write(cr)

        # Compute the ROC curve and AUC for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(len(class_names)):
            fpr[i], tpr[i], _ = roc_curve(y_true == i, y_pred[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Plot the ROC curve for each class
        plt.figure(figsize=(8, 6))
        for i in range(len(class_names)):
            plt.plot(
                fpr[i],
                tpr[i],
                label=f"ROC curve of class {class_names[i]} (AUC = {roc_auc[i]:.2f})",
            )
        plt.plot([0, 1], [0, 1], "k--")
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"ROC Curve for {architecture}_{optimizer_name}")
        plt.legend(loc="lower right")
        plt.tight_layout()
        plt.savefig(os.path.join(visualizations_dir, f"roc_curve_{architecture}_{optimizer_name}.png"))
        plt.close()

        # Visualize the model's predictions on a subset of the validation data
        subset_size = 10
        subset_indices = np.random.choice(len(val_dataset), subset_size, replace=False)
        subset_images = []
        subset_labels = []
        val_dataset.reset()  # Reset the validation dataset iterator
        for i in range(len(val_dataset)):
            if i in subset_indices:
                image_batch, label_batch = next(val_dataset)
                for image, label in zip(image_batch, label_batch):
                    subset_images.append(image)
                    subset_labels.append(label)

        subset_images = np.array(subset_images)
        subset_labels = np.array(subset_labels)

        subset_preds = model.predict(subset_images)
        subset_pred_classes = np.argmax(subset_preds, axis=1)

        # Generate the visualization plot
        plt.figure(figsize=(15, 10))
        for i in range(subset_size):
            plt.subplot(2, 5, i + 1)
            plt.imshow(subset_images[i])
            plt.title(
                f"True: {class_names[np.argmax(subset_labels[i])]}\\nPred: {class_names[subset_pred_classes[i]]}"
            )
            plt.axis("off")
        plt.tight_layout()
        plt.savefig(os.path.join(visualizations_dir, f"predictions_{architecture}_{optimizer_name}.png"))
        plt.close()


# Read the CSV file and store the model results
model_results = []
with open("model_results.csv", "r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        model_results.append(row)

# Create a bar plot for validation precision
plt.figure(figsize=(10, 6))
plt.bar(range(len(model_results)), [float(result["precision"]) for result in model_results])
plt.xticks(
    range(len(model_results)),
    [f"{result['architecture']}_{result['optimizer']}" for result in model_results],
    rotation=45,
)
plt.xlabel("Model")
plt.ylabel("Validation Precision")
plt.title("Validation Precision for Different Models")
plt.tight_layout()
plt.savefig(os.path.join(visualizations_dir, "validation_precision_comparison.png"))
plt.close()

# Create a bar plot for validation recall
plt.figure(figsize=(10, 6))
plt.bar(range(len(model_results)), [float(result["recall"]) for result in model_results])
plt.xticks(
    range(len(model_results)),
    [f"{result['architecture']}_{result['optimizer']}" for result in model_results],
    rotation=45,
)
plt.xlabel("Model")
plt.ylabel("Validation Recall")
plt.title("Validation Recall for Different Models")
plt.tight_layout()
plt.savefig(os.path.join(visualizations_dir, "validation_recall_comparison.png"))
plt.close()

# Create a bar plot for validation AUC-ROC
plt.figure(figsize=(10, 6))
plt.bar(range(len(model_results)), [float(result["auc"]) for result in model_results])
plt.xticks(
    range(len(model_results)),
    [f"{result['architecture']}_{result['optimizer']}" for result in model_results],
    rotation=45,
)
plt.xlabel("Model")
plt.ylabel("Validation AUC-ROC")
plt.title("Validation AUC-ROC for Different Models")
plt.tight_layout()
plt.savefig(os.path.join(visualizations_dir, "validation_auc_comparison.png"))
plt.close()

Found 2003 images belonging to 7 classes.


ValueError: Unknown loss function: 'weighted_categorical_crossentropy'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

# Creating Visualizations Outside of Loop

In [ ]:
# Importing Dependencies
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger

import csv
import os

from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import os


visualizations_dir = "visualizations"

## Read the CSV file and store the model results
model_results = []
with open("model_results.csv", "r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        model_results.append(row)

# Create a bar plot for validation precision
plt.figure(figsize=(10, 6))
plt.bar(range(len(model_results)), [float(result["precision"]) for result in model_results])
plt.xticks(
    range(len(model_results)),
    [f"{result['architecture']}_{result['optimizer']}" for result in model_results],
    rotation=45,
)
plt.xlabel("Model")
plt.ylabel("Validation Precision")
plt.title("Validation Precision for Different Models")
plt.tight_layout()
plt.savefig(os.path.join(visualizations_dir, "validation_precision_comparison.png"))
plt.close()

# Create a bar plot for validation recall
plt.figure(figsize=(10, 6))
plt.bar(range(len(model_results)), [float(result["recall"]) for result in model_results])
plt.xticks(
    range(len(model_results)),
    [f"{result['architecture']}_{result['optimizer']}" for result in model_results],
    rotation=45,
)
plt.xlabel("Model")
plt.ylabel("Validation Recall")
plt.title("Validation Recall for Different Models")
plt.tight_layout()
plt.savefig(os.path.join(visualizations_dir, "validation_recall_comparison.png"))
plt.close()

# Create a bar plot for validation AUC-ROC
plt.figure(figsize=(10, 6))
plt.bar(range(len(model_results)), [float(result["auc"]) for result in model_results])
plt.xticks(
    range(len(model_results)),
    [f"{result['architecture']}_{result['optimizer']}" for result in model_results],
    rotation=45,
)
plt.xlabel("Model")
plt.ylabel("Validation AUC-ROC")
plt.title("Validation AUC-ROC for Different Models")
plt.tight_layout()
plt.savefig(os.path.join(visualizations_dir, "validation_auc_comparison.png"))
plt.close()